In [ ]:
!pip install datasets
!pip install sentence_transformers
!pip install pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import pinecone

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Recommendation/data_file.csv')
dataset.head()

,ID,ProductName,Price,Color,Description
0,1847ddf8-5184-4691-a016-5d4b6c734bb5,Heritage Regular Logo unisex round neck short ...,895000,Green,"Color: White, Black, Green Bay\nMaterial: 63% ..."
1,30547553-03e1-42a3-ac70-5832ce654a2d,Heritage Regular Logo unisex round neck short ...,895000,Black,"Color: White, Black, Green Bay\nMaterial: 63% ..."
2,64786358-ee49-44bb-a56b-9b2ee76986df,Heritage Regular Logo unisex round neck short ...,895000,Black,"Color: White, Black, Green Bay\nMaterial: 63% ..."
3,6a7b6799-569a-457d-a65e-ed43919bd1f3,Heritage Regular Logo unisex round neck short ...,895000,Black,"Color: White, Black, Green Bay\nMaterial: 63% ..."
4,7b89d2c8-bcc0-431b-b682-670bc2d3c828,Heritage Regular Logo unisex round neck short ...,895000,Green,"Color: White, Black, Green Bay\nMaterial: 63% ..."


In [ ]:
dataset['Description'] = dataset['Description'].str.lower()

dataset.head()

,ID,ProductName,Price,Color,Description
0,1847ddf8-5184-4691-a016-5d4b6c734bb5,Heritage Regular Logo unisex round neck short ...,895000,Green,"color: white, black, green bay\nmaterial: 63% ..."
1,30547553-03e1-42a3-ac70-5832ce654a2d,Heritage Regular Logo unisex round neck short ...,895000,Black,"color: white, black, green bay\nmaterial: 63% ..."
2,64786358-ee49-44bb-a56b-9b2ee76986df,Heritage Regular Logo unisex round neck short ...,895000,Black,"color: white, black, green bay\nmaterial: 63% ..."
3,6a7b6799-569a-457d-a65e-ed43919bd1f3,Heritage Regular Logo unisex round neck short ...,895000,Black,"color: white, black, green bay\nmaterial: 63% ..."
4,7b89d2c8-bcc0-431b-b682-670bc2d3c828,Heritage Regular Logo unisex round neck short ...,895000,Green,"color: white, black, green bay\nmaterial: 63% ..."


#### Chunking

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2')
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
def chunk_text(text, chunk_size=10):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

In [ ]:
# Function to vectorize a long text by chunking
def vectorize_long_text(long_text, model, chunk_size=10):
    chunks = chunk_text(long_text, chunk_size)
    chunk_embeddings = model.encode(chunks, convert_to_tensor=True, device=device)
    combined_vector = torch.mean(chunk_embeddings, dim=0).cpu().numpy()
    return combined_vector

In [ ]:
# Function to vectorize a long text by chunking to color and price

import torch

def vector_des_color_price(des_list, color_list, price_list):
    # Convert lists to tensors
    des_tensor = torch.tensor(des_list)
    color_tensor = torch.tensor(color_list)
    price_tensor = torch.tensor(price_list)

    combined_tensor = torch.cat((des_tensor, color_tensor, price_tensor), dim=-1)
    combined_vector = torch.mean(combined_tensor, dim=-1)
    combined_vector_numpy = combined_vector.cpu().numpy()

    return combined_vector_numpy



#### Model

In [ ]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = 'd9ba3ead-959b-40ff-985b-9532080af87a'

# configure client
pc = Pinecone(api_key=api_key)


In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = 'sematic-search'

In [ ]:
# Define the index specifications
spec = {
    'metric': 'cosine',
    'dimension': model.get_sentence_embedding_dimension(),
    'pods': 1,
    'replicas': 1,
    'shards': 1,
    'pod_type': 'p1'
}

In [ ]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
query = 'which city is the most populated in the world?'

xq = model.encode(query)
xq.shape

(384,)

In [ ]:
import torch

def vector_des_color_price(des_list, color_list, price_list, name_list):
    # Convert lists to tensors
    des_tensor = torch.tensor(des_list)
    color_tensor = torch.tensor(color_list)
    price_tensor = torch.tensor(price_list)
    name_tensor = torch.tensor(name_list)

    # Ensure that all tensors have the same dimension
    if name_tensor.shape[1] != des_tensor.shape[1] != color_tensor.shape[1] or des_tensor.shape[1] != price_tensor.shape[1]:
        raise ValueError("All input tensors must have the same number of dimensions.")

    # Compute the average of the tensors along the first dimension
    combined_tensor = torch.mean(torch.stack((des_tensor, color_tensor, price_tensor, name_tensor)), dim=0)

    return combined_tensor

In [ ]:
from tqdm.auto import tqdm

# Define batch size and vector limit
batch_size = 128
vector_limit = 20

# Get descriptions, color, and price from dataset
descriptions = dataset['Description'].tolist()
colors = dataset['Color'].tolist()
prices = dataset['Price'].astype(str).tolist()
productName = dataset['ProductName'].tolist()
id = dataset['ID'].tolist()
# Iterate over descriptions in batches
for i in tqdm(range(0, len(descriptions), batch_size)):
    i_end = min(i + batch_size, len(descriptions))
    ids = [str(x) for x in range(i, i_end)]
    metadatas = [{'Product Name': productName[j],
                  'Description': descriptions[j],
                  'Color' : colors[j],
                  'Price': prices[j],
                  'sku_id': id[j] } for j in range(i, i_end)]

    vectorized_batch = [vectorize_long_text(text, model) for text in productName[i:i_end]]
    color_vector = model.encode(colors[i:i_end])
    price_vector = model.encode(prices[i:i_end])
    name_vector = model.encode(productName[i:i_end])


    # Ensure vectors are converted to lists before passing to vector_des_color_price
    vectorized_batch_list = [v.tolist() for v in vectorized_batch]
    color_vector_list = color_vector.tolist()
    price_vector_list = price_vector.tolist()
    name_vector_list = name_vector.tolist()

    combined_vector = vector_des_color_price(vectorized_batch_list, color_vector_list, price_vector_list, name_vector_list)

    records = [(ids[j], combined_vector[j].tolist(), metadatas[j]) for j in range(len(ids))]

    index.upsert(vectors=records)

# Check number of records in the index
index.describe_index_stats()

  0%|          | 0/1 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
query = "unisex round neck short in black"

xq = model.encode(query).tolist()

xc = index.query(vector=xq, top_k=5, include_metadata=True)
xc

{'matches': [{'id': '42',
              'metadata': {'Color': 'Black',
                           'Description': 'the design with the brand logo '
                                          'printed large on the back and a '
                                          'small logo printed on the front '
                                          'creates an attractive touch for '
                                          'this t-shirt. more specifically, '
                                          'with a comfortable loose design and '
                                          'airy material, this shirt will give '
                                          'you an extremely comfortable '
                                          'feeling when wearing it, definitely '
                                          'a must-have-item that you cannot '
                                          'ignore. .\n'
                                          'brand: fila\n'
                                   